<center><h1>POD-Mini: minimized proof of delivery</h1></center>
<center><h2>implemented via Python</h2><center>

## Intro

This is a Python implementaton of Pod-Mini of [zkPod](https://github.com/sec-bit/zkPoD-node) via klefki, for more details, just check the [technial Paper](https://github.com/sec-bit/zkPoD-node).

In [1]:
from klefki.types.algebra.concrete import (
    EllipticCurveGroupSecp256k1 as ECG,
    EllipticCurveCyclicSubgroupSecp256k1 as CG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)


G = CG.G

In [17]:
import random

N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
random_f = lambda: CF(random.randint(1, N) % CF.P)

q = random_f()
H = G @ q
    

In [18]:
from hashlib import sha256

hash = lambda x, y: CF(int(sha256(str(x.value + y).encode()).hexdigest(), 16) % N)

## Initializer Phase

In [19]:
data = 123456789
tag = 6

In [22]:
m = CF(data)
o = CF(tag)

In [23]:
sigma = G @ m + H @ o

## Deliver Phase

In [24]:
k_w = random_f()

k = hash(k_w, 1)
k_ = hash(k_w, 2)
k0 = hash(k_w, 3)
k0_ = hash(k_w, 4)

In [25]:
K = G @ k + H @ k_
K0 = G @ k0 + H @ k0_
(K, K0)

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::4192133894987479521686609513582066668531014381843683945302788755767037189152, FiniteFieldSecp256k1::104216824248948063900856029232918945386834245436458613611954254638255878359112),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::84911789591068450313882454388126163502581911102858115020166344656333657793244, FiniteFieldSecp256k1::67495519763398757602804923572938985582440954931552251292769498802538287538334))

* $S \rightarrow R$: $(K, K_0)$

In [26]:
c = random_f()

* $R \rightarrow S$: $c$

In [27]:
m_ = k + c * m
o_ = k_ + c * o
z = k0 + c * k
z_ = k0_ + c * k_
(m_, o_, z, z_)

(FiniteFieldCyclicSecp256k1::111753046838732497637051533132157354014452335816056776976503298388519138824896,
 FiniteFieldCyclicSecp256k1::109803129159021823841277671733826972348060620797058786622158808107345148035543,
 FiniteFieldCyclicSecp256k1::68192318912878036677560258532401015223913045976096603326050271134237807559694,
 FiniteFieldCyclicSecp256k1::15311673516255252339494668905660435400990366245688893192147427968829657181944)

* $S \rightarrow R: (\bar{m}, \bar{o}, z, z')$

R should verify that:

$$
Com(m; o)^c \cdot Com(k_0;k'_0) \stackrel{?}{=} Com(\bar{m}; \bar{o})\\
Com(k_0;k'_0) \cdot Com(k;k')^c \stackrel{?}{=} Com(z;z')
$$

In [33]:
assert sigma @ c + K == G @ m_ + H @ o_

In [34]:
assert K0 + (K @ c) == G @ z + H @ z_

### Reveal-phase

* $R \rightarrow J: \rho$
* $S \rightarrow J: k_{\omega}$

$$
z \stackrel{?}{=} H(k_{\omega}, 3) + c \cdot H(k_{\omega}, 1)
$$

In [35]:
assert z == hash(k_w, 3) + c @ hash(k_w, 1)

In [36]:
assert m == (m_ - hash(k_w, 1)) / c

In [37]:
(m_ - hash(k_w, 1)) / c

FiniteFieldCyclicSecp256k1::123456789